In [132]:
import pandas as pd
import numpy as np
import time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor

In [96]:
dfValue = pd.read_csv('train.csv',engine='c',memory_map=True)

In [97]:
dfTest = pd.read_csv('test.csv',engine='c',memory_map=True)

In [98]:
X = dfValue.loc[:, dfValue.columns != 'trip_duration']
Y = dfValue.trip_duration

In [99]:
pickupDatetime  = pd.to_datetime(X.pickup_datetime)
dropoffDatetime = pd.to_datetime(X.dropoff_datetime)

XDrop = X.drop(['id','pickup_datetime','dropoff_datetime'],axis=1)

# On reccréé des colonnes de pickup en séparant les informations de la date
XDrop['pickupHour'] = pickupDatetime.dt.hour
XDrop['pickupDayOfWeek'] = pickupDatetime.dt.dayofweek
XDrop['pickupWeekOfYear'] = pickupDatetime.dt.weekofyear
XDrop['pickupDayOfYear'] = pickupDatetime.dt.dayofyear

# Pareil pour le dropoff
XDrop['dropoffHour'] = dropoffDatetime.dt.hour
XDrop['dropoffDayOfWeek'] = dropoffDatetime.dt.dayofweek
XDrop['dropoffWeekOfYear'] = dropoffDatetime.dt.weekofyear
XDrop['dropoffDayOfYear'] = dropoffDatetime.dt.dayofyear

XTrain, XTest, YTrain, Ytest = train_test_split(XDrop,Y,train_size=0.85)

In [100]:
#pipeline = Pipeline(steps=[('preprocessor', preprocessor),('model', model)]).fit(XTrain,YTrain)


## PREPROCESSING LABEL ENCODER ##
# labelEncoder = LabelEncoder()
# XTrainFinal = XTrain.copy()
# XTestFinal = XTest.copy()

# XTrainFinal.store_and_fwd_flag = labelEncoder.fit_transform(XTrain.store_and_fwd_flag)
# XTestFinal.store_and_fwd_flag = labelEncoder.transform(XTest.store_and_fwd_flag)

## PREPROCESSING ONE HOT ENCODER
OHEncoder = OneHotEncoder(sparse=False)
OHColumnsTrain = pd.DataFrame(OHEncoder.fit_transform(XTrain[['store_and_fwd_flag']]))
OHColumnsTest  = pd.DataFrame(OHEncoder.transform(XTest[['store_and_fwd_flag']]))

OHColumnsTrain.index = XTrain.index
OHColumnsTest.index = XTest.index

XTrainCategoricalRemoved = XTrain.loc[:, XTrain.columns != 'store_and_fwd_flag']
XTestCategoricalRemoved = XTest.loc[:, XTest.columns != 'store_and_fwd_flag']

XTrainFinal = pd.concat([XTrainCategoricalRemoved, OHColumnsTrain], axis=1)
XTestFinal = pd.concat([XTestCategoricalRemoved, OHColumnsTest], axis=1)

In [137]:
#n_estimators=150 max_depth=10 --> meilleur résultats pour 10_000 lignes avec OHEncoder
print(XTrainFinal.shape, ' ', XTestFinal.shape)
n = 30
for d in [10,20,30,40,50,60,70]:
    print ("n=",n," d=", d,' ',sep='')
    model = RandomForestRegressor(random_state=19061996,n_estimators=n, max_depth=d)
    start = time.time()
    model.fit(XTrainFinal,YTrain)
    end = time.time()
    preds = model.predict(XTestFinal)
    mean = mean_absolute_error(Ytest,preds)
    print("t=",(end-start),"s e=",mean,sep='')


(1239847, 16)   (218797, 16)
n=30 d=10 
